This Notebook will model the rear irradiance for 200 sensors across a 1 panel system, with torquetube, 

In [1]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
    
import numpy as np

In [ ]:
# The below routine creates a HEXAGONAL torque tube, for a 2-UP configuration of a specific module size. Parameters for the module, the torque tube, and the scene are below.
# This is being run with gendaylit, for one specific timestamp
# MODIFY THE TEST FOLDER TO YOUR LIKING. I utmostly dislike the interactive directory.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo4'

timestamp = 4020 # Noon, June 17th. 
simulationname = 'Regular_Array'

# MakeModule Parameters
moduletype=simulationname
numpanels = 1
x = 1  
y = 2
xgap = 0
ygap = 0
zgap = 0
sensorsy = 200 

# TorqueTube Parameters
axisofrotationTorqueTube=False
torqueTube = False
tubetype = 'square'
diameter = 0.20
torqueTubeMaterial = 'black'       # IT's NOT GRAY, IT's GREY
cellLevelModule = False
numcellsx=0
numcellsy=0

# SceneDict Parameters
gcr = 0.33   # ground cover ratio,  = module_height / pitch
albedo = 0.28  #'concrete'     # ground albedo
clearance_height = 0.5  
nMods = 20
nRows = 7
azimuth_ang=180
tilt =30

# Now let's run the example

demo = RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(0.62) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo.readEPW(epwfile) # read in the EPW weather data from above

# Making module with all the variables
moduleDict=demo.makeModule(name=moduletype,x=x,y=y,bifi=1, 
           torquetube=torqueTube, diameter = diameter, tubetype = tubetype, 
                           material = torqueTubeMaterial, zgap = zgap, numpanels = numpanels, ygap = ygap, 
                           rewriteModulefile = True, xgap=xgap, 
                           axisofrotationTorqueTube=axisofrotationTorqueTube, cellLevelModule=cellLevelModule, 
                           numcellsx=numcellsx, numcellsy = numcellsy)
# create a scene with all the variables
sceneDict = {'tilt':tilt,'pitch': 3,'height':clearance_height,'azimuth':azimuth_ang, 'module_type':simulationname, 'nMods': nMods, 'nRows': nRows}  
scene = demo.makeScene(moduletype=moduletype, sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.

for timeindex in range (1416,2160):
    demo.gendaylit(metdata,timeindex)  # Noon, June 17th
    analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance

    frontscan, backscan = analysis.moduleAnalysis(sceneDict['height'], sceneDict['azimuth'], sceneDict['tilt'], 
                                              sceneDict['pitch'], sceneDict['nMods'], sceneDict['nRows'], 
                                              moduleDict['sceney'], moduleDict['scenex'], moduleDict['moduleoffset'],
                                              sensorsy=sensorsy)
    analysis.analysis(octfile, "Shading_"+str(timeindex), frontscan, backscan)  # compare the back vs front irradiance  

path = C:\Users\sayala\Documents\RadianceScenes\Demo4
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
('\nModule Name:', 'Regular_Array')
REWRITING pre-existing module file. 
Module Regular_Array successfully created
Created Regular_Array.oct
('modWanted, rowWanted ', 10.0, 4.0, ' out of ', 20, 7)
Linescan in process: Shading_1416_Front
message: rtrace: warning - no light sources found
Linescan in process: Shading_1416_Back
message: rtrace: warning - no light sources found
Saved: results\irr_Shading_1416.csv
('modWanted, rowWanted ', 10.0, 4.0, ' out of ', 20, 7)
Linescan in process: Shading_1417_Front
message: rtrace: warning - no light sources found
Linescan in process: Shading_1417_Back
message: rtrace: warning - no light sources found
Saved: results\irr_Shading_1417.csv
('modWanted, rowWanted ', 10.0, 4.0, ' out of ', 20, 7)
Linescan in process: Shading_1418_Front
message: rtrace: warning - no light sources found
Linescan in process: Shading_1418_Back
message: